<a href="https://colab.research.google.com/github/newmantic/parallel_prefix/blob/main/parallel_prefix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from concurrent.futures import ThreadPoolExecutor

In [2]:
def parallel_prefix_sum(arr, inclusive=True):
    n = len(arr)
    if n == 0:
        return arr

    output = np.zeros(n, dtype=arr.dtype)

    if inclusive:
        # Start with inclusive scan (each element includes itself in the sum)
        output[0] = arr[0]
    else:
        # Start with exclusive scan (first element is 0, sum starts from second element)
        output[0] = 0

    def compute_range(start, end, step):
        for i in range(start, end, step):
            output[i] = output[i - step] + arr[i - step]

    # Phase 1: Up-sweep (reduce) phase
    step = 1
    while step < n:
        with ThreadPoolExecutor() as executor:
            for i in range(step, n, 2 * step):
                executor.submit(compute_range, i, n, step)
        step *= 2

    if not inclusive:
        # Phase 2: Down-sweep phase for exclusive scan
        output[-1] = 0
        step //= 2
        while step > 0:
            with ThreadPoolExecutor() as executor:
                for i in range(step, n, 2 * step):
                    executor.submit(compute_range, i, n, step)
            step //= 2

        # Correct final step for exclusive scan
        for i in range(n - 1, 0, -1):
            output[i] = output[i - 1] + arr[i - 1]
        output[0] = 0

    return output

In [3]:
def test_case_1():
    arr = np.array([1, 2, 3, 4, 5])
    result = parallel_prefix_sum(arr, inclusive=True)
    print("Input array:", arr)
    print("Inclusive Prefix Sum:", result)

test_case_1()

Input array: [1 2 3 4 5]
Inclusive Prefix Sum: [1 2 2 7 2]


In [4]:
def test_case_2():
    arr = np.array([1, 2, 3, 4, 5])
    result = parallel_prefix_sum(arr, inclusive=False)
    print("Input array:", arr)
    print("Exclusive Prefix Sum:", result)

test_case_2()

Input array: [1 2 3 4 5]
Exclusive Prefix Sum: [ 0  1  3  6 10]


In [5]:
def test_case_3():
    arr = np.random.randint(1, 100, size=1000)
    result_inclusive = parallel_prefix_sum(arr, inclusive=True)
    result_exclusive = parallel_prefix_sum(arr, inclusive=False)
    print("First 10 elements of the input array:", arr[:10])
    print("First 10 elements of the Inclusive Prefix Sum:", result_inclusive[:10])
    print("First 10 elements of the Exclusive Prefix Sum:", result_exclusive[:10])

test_case_3()

First 10 elements of the input array: [35 11 32 11 97 18 50 49 90 61]
First 10 elements of the Inclusive Prefix Sum: [ 35  70  70 113  70 221 199 289  70 428]
First 10 elements of the Exclusive Prefix Sum: [  0  35  46  78  89 186 204 254 303 393]
